In [1]:
import torch
import pickle
import pandas as pd

In [2]:
## loading the encoder and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_got_encoder_geo.pkl','rb') as file:
    one_hot_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [12]:
input_data ={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [13]:
geo_encoded = one_hot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

geo_encoded_df

d:\Projects\ann-classfication\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [14]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [15]:
input_df  = pd.concat([input_df.reset_index(drop=True),geo_encoded_df],axis=1)
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [16]:
input_df['Gender'] = label_encoder_gender.transform([input_data['Gender']])

In [17]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [18]:
input_df = input_df.drop(columns=['Geography'])

In [19]:
input_df.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [20]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [21]:
features = torch.tensor(input_scaled,dtype=torch.float32)
features

tensor([[-0.5360,  0.9132,  0.1048, -0.6954, -0.2578,  0.8084,  0.6492,  0.9748,
         -0.8768,  1.0015, -0.5795, -0.5764]])

In [22]:

model_script = torch.jit.load('model_script.pt')
model_script.eval() 

RecursiveScriptModule(
  original_name=Net
  (layer1): RecursiveScriptModule(original_name=Linear)
  (layer2): RecursiveScriptModule(original_name=Linear)
  (layer3): RecursiveScriptModule(original_name=Linear)
)

In [23]:
with torch.no_grad():
    output = model_script(features)
    print(f"Output : {output.item()}")

Output : 0.17256423830986023


In [24]:
if output.item()>0.5:
    print('Customer is likely to churn')
else:
    print('Customer is unlikely to churn')

Customer is unlikely to churn
